Постройте программу, которая в качестве входа принимает приведенную КС-грамматику G и
преобразует ее в эквивалентную КС-грамматику G' без левой рекурсии.

In [170]:
from functools import reduce 

class GrammarRule:
    def __init__(self, l: str, r:list[str]) -> None:
        self.left = l
        self.right = r.copy()

class Grammar:
    nt = []
    term = []
    rules = []
    axiom = ''

    def __init__(self, nt: list[str], term: list[str], rules: list[GrammarRule], axiom: str) -> None:
        self.nt = nt
        self.term = term
        self.rules = rules
        self.axiom = axiom
    
    def print(self, std=True, md=True):
        output = ""
        output += "[{:}] [{:}] {:}\n".format(",".join(sorted(self.nt)), ",".join(sorted(self.term)), self.axiom) 

        for nt in sorted(self.nt):
            rules = list(filter(lambda x: x.left == nt, self.rules))
            if not len(rules):
                continue
            
            output += '\n' + ( nt + " -> " + " | ".join(sorted(map(lambda x: " ".join(x.right) if len(x.right) else '\epsilon', rules))))
        if std:
            print(output)
        
        if md:
            output = "\n\n".join(map(lambda x: '$' + x.replace('->', '\\rightarrow') + '$' if len(x) else '', output.split('\n')))

        return output
    def show(self):
        import IPython

        return IPython.display.Markdown(self.print(std=False))
    @staticmethod
    def from_text(text):
        nt, t, a = text.split('\n')[0].split(' ')
        nt = nt[1:-1].split(',')
        t = t[1:-1].split(',')
        
        rules = []

        for string in text.split('\n')[1:]:
            if not len(string):
                continue
            
            left, right = string.split(' -> ')

            for rule in right.split(' | '):
                data = rule.split(' ')
                if data[0] == '\epsilon':
                    data = []
                rules.append(
                    GrammarRule(left, data)
                )
        return Grammar(nt, t, rules, a)



In [173]:
grammar = Grammar(['E', 'T', 'F', 'G', 'R', 'U'], ['+', '-', '(', ')', 'a'], [
    GrammarRule('E', ['E', '+', 'T']),
    GrammarRule('E', ['T']),
    GrammarRule('T', ['T', '*', 'F']),
    GrammarRule('T', ['F']),
    GrammarRule('F', ['a']),
    GrammarRule('F', ['G']),
    GrammarRule('G', ['F']),
    GrammarRule('G', ['G', '+', 'R']),
    GrammarRule('G', []),
    GrammarRule('F', ['(', 'E', ')']),
    GrammarRule('U', ['F']),
], 'E')

grammar.show()

$[E,F,G,R,T,U] [(,),+,-,a] E$



$E \rightarrow E + T | T$

$F \rightarrow ( E ) | G | a$

$G \rightarrow F | G + R | \epsilon$

$T \rightarrow F | T * F$

$U \rightarrow F$

In [175]:
def remove_left_recursion(g: Grammar):
    new_rules: list[GrammarRule] = g.rules.copy()

    nts = g.nt.copy()


    for i, i_nt in enumerate(g.nt):
        for j_nt in g.nt[:i]:
            rules = filter(lambda x: x.left == i_nt and len(x.right) and x.right[0] == j_nt, new_rules)
            for rule in rules:
                new_rules.remove(rule)
                m_rules = filter(lambda x: x.left == j_nt, new_rules)
                for m_rule in m_rules:
                    new_rules.append(
                        GrammarRule(
                            i_nt, m_rule.right + rule.right[1:]
                        )
                    )
        rules = list(filter(lambda x: x.left == i_nt, new_rules))
        need_modify = False
        for rule in rules:
            if len(rule.right) and rule.right[0] == i_nt:
                need_modify = True
                break
                
        if need_modify:
            new_nt = i_nt + "'"
            nts += [new_nt]

            for rule in rules:
                new_rules.remove(rule)
                if len(rule.right) and rule.right[0] == i_nt:
                    new_rules += [
                        GrammarRule(new_nt, rule.right[1:]),
                        GrammarRule(new_nt, rule.right[1:] + [new_nt]),
                    ]
                else:
                    new_rules += [
                        GrammarRule(rule.left, rule.right.copy()),
                        GrammarRule(rule.left, rule.right + [new_nt]),
                    ]
        


    
    return Grammar(nts, g.term.copy(), new_rules, g.axiom)


  
display(grammar.show())
print('='*10)
remove_left_recursion(grammar).show()

$[E,F,G,R,T,U] [(,),+,-,a] E$



$E \rightarrow E + T | T$

$F \rightarrow ( E ) | G | a$

$G \rightarrow F | G + R | \epsilon$

$T \rightarrow F | T * F$

$U \rightarrow F$

$[E,E',F,G,G',R,T,T',U] [(,),+,-,a] E$



$E \rightarrow T | T E'$

$E' \rightarrow + T | + T E'$

$F \rightarrow ( E ) | G | a$

$G \rightarrow ( E ) | ( E ) G' | G' | \epsilon | a | a G'$

$G' \rightarrow + R | + R G' | G' | \epsilon$

$T \rightarrow F | F T'$

$T' \rightarrow * F | * F T'$

$U \rightarrow ( E ) | ( E ) | ( E ) G' | G' | \epsilon | a | a | a G'$

Вариант 2. Устранение бесполезных символов.

Постройте программу, которая в качестве входа принимает произвольную КС-грамматику G  и
преобразует ее в эквивалентную КС-грамматику G', не содержащую бесполезных символов.

Алгоритм состоит из двух этапов:
- Удалить из грамматики правила, содержащие непорождающие нетерминалы.
- Удалить из грамматики правила, содержащие недостижимые нетерминалы.

In [176]:
def remove_notgener_symbols(g: Grammar):
    nt = set([g.axiom])

    modified = True
    while modified:    
        def if_onlyterm(rule: GrammarRule):
            if not len(rule.right):
                return False
            
            for x in rule.right:
                if not (x in g.term or x in nt):
                    return False
                    
            return True

        added = set(map(lambda rule: rule.left, filter(if_onlyterm, g.rules)))
        new_nt = nt.union(added)
        if len(new_nt) == len(nt):
            modified = False

        nt = nt.union(added)
    
    rules = [
        rule for rule in g.rules 
        if rule.left in nt and 
        reduce(lambda x, acc: acc and x, map(lambda x: x in nt or x in g.term, rule.right), True)
    ]

    return Grammar(nt, g.term, rules, g.axiom)

display(grammar.show())
print("====")
remove_notgener_symbols(grammar).show()



$[E,F,G,R,T,U] [(,),+,-,a] E$



$E \rightarrow E + T | T$

$F \rightarrow ( E ) | G | a$

$G \rightarrow F | G + R | \epsilon$

$T \rightarrow F | T * F$

$U \rightarrow F$

====


$[E,F,G,T,U] [(,),+,-,a] E$



$E \rightarrow E + T | T$

$F \rightarrow ( E ) | G | a$

$G \rightarrow F | \epsilon$

$T \rightarrow F$

$U \rightarrow F$

In [178]:
def remove_notfollow_symbols(g: Grammar):
    queue = [g.axiom]
    nts = [ g.axiom]

    new_rules = []

    while len(queue):
        nt = queue.pop(0)

        for rule in filter(lambda x: x.left == nt, g.rules):
            new_rules.append(rule)
            for symbol in rule.right:
                if symbol in g.nt and symbol not in nts:
                    nts.append(symbol)
                    queue.append(symbol)

    return Grammar(nts, g.term, new_rules, g.axiom)

    

display(grammar.show())
print("====")
remove_notfollow_symbols(grammar).show()

$[E,F,G,R,T,U] [(,),+,-,a] E$



$E \rightarrow E + T | T$

$F \rightarrow ( E ) | G | a$

$G \rightarrow F | G + R | \epsilon$

$T \rightarrow F | T * F$

$U \rightarrow F$

====


$[E,F,G,R,T] [(,),+,-,a] E$



$E \rightarrow E + T | T$

$F \rightarrow ( E ) | G | a$

$G \rightarrow F | G + R | \epsilon$

$T \rightarrow F | T * F$

In [148]:
grammar.show()
print("====")
remove_left_recursion(grammar).show()
print("====")
remove_notgener_symbols(remove_left_recursion(grammar)).show()
print("====")
remove_notfollow_symbols(remove_notgener_symbols(remove_left_recursion(grammar))).show()


[E,F,G,R,T,U] [(,),+,-,a] E

E -> E + T | T
F -> ( E ) | G | a
G -> F | G + R | \eps
T -> F | T * F
U -> F
====
[E,E',F,G,G',R,T,T',U] [(,),+,-,a] E

E -> T | T E'
E' -> + T | + T E'
F -> ( E ) | G | a
G -> ( E ) | ( E ) G' | G' | \eps | a | a G'
G' -> + R | + R G' | G' | \eps
T -> F | F T'
T' -> * F | * F T'
U -> ( E ) | ( E ) | ( E ) G' | G' | \eps | a | a | a G'
====
[E,E',F,G,T,U] [(,),+,-,a] E

E -> T | T E'
E' -> + T | + T E'
F -> ( E ) | G | a
G -> ( E ) | \eps | a
T -> F
U -> ( E ) | ( E ) | \eps | a | a
====
[E,E',F,G,T] [(,),+,-,a] E

E -> T | T E'
E' -> + T | + T E'
F -> ( E ) | G | a
G -> ( E ) | \eps | a
T -> F


"$[E,E',F,G,T] [(,),+,-,a] E$\n\n\n\n$E \\rightarrow T | T E'$\n\n$E' \\rightarrow + T | + T E'$\n\n$F \\rightarrow ( E ) | G | a$\n\n$G \\rightarrow ( E ) | \\eps | a$\n\n$T \\rightarrow F$"

In [149]:
import IPython

def cmp_g(l_r, r_g):
    return l_r.print(False) == r_g.print(False)

def test(func, tests, full=False):
    output = "<table><thead><tr><td>Вход</td><td>Ожидаемый результат</td><td>Действительный результат</td><td>Комментарий</td></tr></thead><tbody>"
    for test in tests:
        l = func(test[0])
        res = cmp_g(l, test[1])
        if not res or full:
            output += "<tr><td>\n\n{:}\n</td><td>\n\n{:}\n</td><td>\n\n{:}</td><td>\n\n{:}\n</td></tr>\n".format(
                test[0].print(std=False),
                test[1].print(std=False),
                l.print(std=False),
                "OK" if res else "ERROR"
            )
    output += "</tbody></table>"
    return output

In [150]:
report = test(remove_left_recursion, [
(
Grammar.from_text(
"""[A] [a] A

A -> a A
"""),
Grammar.from_text(
"""[A] [a] A

A -> a A
"""),
),
(
Grammar.from_text(
"""[A] [+,-,a] A

A -> A + A | A - A | a
"""),
Grammar.from_text(
"""[A,A'] [+,-,a] A

A -> a | a A'
A' -> + A | + A A' | - A | - A A'
"""),
),
(
Grammar.from_text(
"""[A,B] [a,b] A

A -> B a | a
B -> A b | b
"""),
Grammar.from_text(
"""[A,B,B'] [a,b] A

A -> B a | a
B -> a b | a b B' | b | b B'
B' -> a b | a b B'
"""),
)
], True)



IPython.display.Markdown(report)

<table><thead><tr><td>Вход</td><td>Ожидаемый результат</td><td>Действительный результат</td><td>Комментарий</td></tr></thead><tbody><tr><td>

$[A] [a] A$



$A \rightarrow a A$
</td><td>

$[A] [a] A$



$A \rightarrow a A$
</td><td>

$[A] [a] A$



$A \rightarrow a A$</td><td>

OK
</td></tr>
<tr><td>

$[A] [+,-,a] A$



$A \rightarrow A + A | A - A | a$
</td><td>

$[A,A'] [+,-,a] A$



$A \rightarrow a | a A'$

$A' \rightarrow + A | + A A' | - A | - A A'$
</td><td>

$[A,A'] [+,-,a] A$



$A \rightarrow a | a A'$

$A' \rightarrow + A | + A A' | - A | - A A'$</td><td>

OK
</td></tr>
<tr><td>

$[A,B] [a,b] A$



$A \rightarrow B a | a$

$B \rightarrow A b | b$
</td><td>

$[A,B,B'] [a,b] A$



$A \rightarrow B a | a$

$B \rightarrow a b | a b B' | b | b B'$

$B' \rightarrow a b | a b B'$
</td><td>

$[A,B,B'] [a,b] A$



$A \rightarrow B a | a$

$B \rightarrow a b | a b B' | b | b B'$

$B' \rightarrow a b | a b B'$</td><td>

OK
</td></tr>
</tbody></table>

In [151]:
report = test(remove_notgener_symbols, [
(
Grammar.from_text(
"""[A] [a] A

A -> a A
"""),
Grammar.from_text(
"""[A] [a] A

A -> a A
"""),
),
(
Grammar.from_text(
"""[A,B] [a,b] A
A -> B | A a | a
B -> B b
"""),
Grammar.from_text(
"""[A] [a,b] A
A -> A a | a
"""),
),
(
Grammar.from_text(
"""[A,B] [a,b] A
A -> B | A a | a
"""),
Grammar.from_text(
"""[A] [a,b] A
A -> A a | a
"""),
),
], True)


IPython.display.Markdown(report)

<table><thead><tr><td>Вход</td><td>Ожидаемый результат</td><td>Действительный результат</td><td>Комментарий</td></tr></thead><tbody><tr><td>

$[A] [a] A$



$A \rightarrow a A$
</td><td>

$[A] [a] A$



$A \rightarrow a A$
</td><td>

$[A] [a] A$



$A \rightarrow a A$</td><td>

OK
</td></tr>
<tr><td>

$[A,B] [a,b] A$



$A \rightarrow A a | B | a$

$B \rightarrow B b$
</td><td>

$[A] [a,b] A$



$A \rightarrow A a | a$
</td><td>

$[A] [a,b] A$



$A \rightarrow A a | a$</td><td>

OK
</td></tr>
<tr><td>

$[A,B] [a,b] A$



$A \rightarrow A a | B | a$
</td><td>

$[A] [a,b] A$



$A \rightarrow A a | a$
</td><td>

$[A] [a,b] A$



$A \rightarrow A a | a$</td><td>

OK
</td></tr>
</tbody></table>

In [152]:
report = test(remove_notfollow_symbols, [
(
Grammar.from_text(
"""[A] [a] A

A -> a A
"""),
Grammar.from_text(
"""[A] [a] A

A -> a A
"""),
),
(
Grammar.from_text(
"""[A,B] [a,b] A
A -> A a | a
B -> A b
"""),
Grammar.from_text(
"""[A] [a,b] A
A -> A a | a
"""),
),
(
Grammar.from_text(
"""[A,B,C] [a,b,c] A
A -> A a | a
B -> C b 
C -> B c
"""),
Grammar.from_text(
"""[A] [a,b,c] A
A -> A a | a
"""),
),
(
Grammar.from_text(
"""[A,B,C] [a,b,c] A
A -> A a | a | B
B -> C b 
C -> B c
"""),
Grammar.from_text(
"""[A,B,C] [a,b,c] A
A -> A a | a | B
B -> C b 
C -> B c
"""),
),

], True)


IPython.display.Markdown(report)

<table><thead><tr><td>Вход</td><td>Ожидаемый результат</td><td>Действительный результат</td><td>Комментарий</td></tr></thead><tbody><tr><td>

$[A] [a] A$



$A \rightarrow a A$
</td><td>

$[A] [a] A$



$A \rightarrow a A$
</td><td>

$[A] [a] A$



$A \rightarrow a A$</td><td>

OK
</td></tr>
<tr><td>

$[A,B] [a,b] A$



$A \rightarrow A a | a$

$B \rightarrow A b$
</td><td>

$[A] [a,b] A$



$A \rightarrow A a | a$
</td><td>

$[A] [a,b] A$



$A \rightarrow A a | a$</td><td>

OK
</td></tr>
<tr><td>

$[A,B,C] [a,b,c] A$



$A \rightarrow A a | a$

$B \rightarrow C b $

$C \rightarrow B c$
</td><td>

$[A] [a,b,c] A$



$A \rightarrow A a | a$
</td><td>

$[A] [a,b,c] A$



$A \rightarrow A a | a$</td><td>

OK
</td></tr>
<tr><td>

$[A,B,C] [a,b,c] A$



$A \rightarrow A a | B | a$

$B \rightarrow C b $

$C \rightarrow B c$
</td><td>

$[A,B,C] [a,b,c] A$



$A \rightarrow A a | B | a$

$B \rightarrow C b $

$C \rightarrow B c$
</td><td>

$[A,B,C] [a,b,c] A$



$A \rightarrow A a | B | a$

$B \rightarrow C b $

$C \rightarrow B c$</td><td>

OK
</td></tr>
</tbody></table>